In [ ]:
import cluster.RayJobManager

In [ ]:
RAY_ADDRESS = "http://10.100.192.238:8265"
ENTRYPOINT  = "python3 cluster/tune.py"

ray_manager = RayJobManager(RAY_ADDRESS)

In [ ]:
job_id = ray_manager.submit_job(
    entrypoint=ENTRYPOINT,
    pip_packages=[
        "torch",
        "git+https://github.com/EmanueleCoradin/TrackSelectorDNN.git@main"
    ]
)

ray_manager.wait_until_done(job_id)